# REddyProc variables

- <https://github.com/bgctw/REddyProc>
- <https://ameriflux.lbl.gov/data/aboutdata/data-variables/>

In [1]:
if (grepl("docs", getwd(), fixed = TRUE)) {
  setwd("..")
}

In [2]:
library(REddyProc)
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [3]:
# REddyProc - AmeriFlux variable name mapping
vars_bgc <- data.frame(getBGC05ToAmerifluxVariableNameMapping())

In [4]:
head(vars_bgc)

,getBGC05ToAmerifluxVariableNameMapping..
,<chr>
Year,YEAR
DoY,DOY
Rg,SW_IN
Tair,TA
Tsoil,TS
rH,RH


In [5]:
# assign column name
names(vars_bgc) <- "Variable"

In [6]:
# convert row names to column
vars_bgc$BGC <- row.names(vars_bgc)

In [7]:
# AmeriFlux vars - downloaded from their website
vars_af <- read.csv("data/flux-met_processing_variables.csv")

In [8]:
head(vars_af)

,Type,Variable,Description,Units
,<chr>,<chr>,<chr>,<chr>
1,TIMEKEEPING,TIMESTAMP_START,ISO timestamp start of averaging period (up to a 12-digit integer as specified by the data's temporal resolution),YYYYMMDDHHMM
2,TIMEKEEPING,TIMESTAMP_END,ISO timestamp end of averaging period (up to a 12-digit integer as specified by the data's temporal resolution),YYYYMMDDHHMM
3,TIMEKEEPING,TIMESTAMP,ISO timestamp (up to a 12-digit integer as specified by the data's temporal resolution),YYYYMMDDHHMM
4,AQUATIC,COND_WATER,"Conductivity (i.e., electrical conductivity) of water",µS cm-1
5,AQUATIC,DO,Dissolved oxygen in water,µmol L-1
6,AQUATIC,PCH4,Dissolved methane (CH4) in water,nmolCH4 mol-1


In [9]:
# merge into a single dataframe with variables relevant to REddyProc
vars_all <- merge(vars_bgc, vars_af, by = "Variable", all.x = TRUE)

In [10]:
vars_all

Variable,BGC,Type,Description,Units
<chr>,<chr>,<chr>,<chr>,<chr>
DOY,DoY,NA,NA,NA
H_F,H_f,NA,NA,NA
H_PI,H_orig,NA,NA,NA
H_QC,H_fqc,NA,NA,NA
LE_F,LE_f,NA,NA,NA
LE_PI,LE_orig,NA,NA,NA
LE_QC,LE_fqc,NA,NA,NA
NEE_F,NEE_f,NA,NA,NA
NEE_PI,NEE_orig,NA,NA,NA


In [11]:
# handle the NAs - use the variable name without the prefix
l <- strsplit(as.character(vars_all[!complete.cases(vars_all),]$Variable), "_")

In [12]:
# create a dataframe
l <- data.frame(
  Variable = unlist(l),
  Var = rep(vars_all[!complete.cases(vars_all),]$Variable, lengths(l))
)

In [13]:
# remove duplicates based on actual var name
l <- distinct(l, Var, .keep_all = TRUE)

In [14]:
# merge with AmeriFlux vars
l <- merge(l, vars_af, by = "Variable", all.x = TRUE)

In [15]:
# switch back to the actual var names
l$Variable <- NULL

In [16]:
names(l)[names(l) == "Var"] <- "Variable"

In [17]:
# merge with main var list
vars_all <- merge(
  merge(l, vars_all[!complete.cases(vars_all),], all.x = TRUE),
  vars_all[complete.cases(vars_all),], all = TRUE
)

In [18]:
# merge again with REddyProc vars
vars_all$BGC <- NULL

In [19]:
vars_all <- merge(vars_all, vars_bgc)

In [20]:
# add info about prefixes
vars_all$Qualifier <- ifelse(
  grepl("_F", vars_all$Variable), "Gap-filled variable",
  ifelse(
    grepl("_PI", vars_all$Variable), "Provided by PI",
    ifelse(grepl("_QC", vars_all$Variable), "Quality control flag", NA)
  )
)

In [21]:
vars_all

Variable,Type,Description,Units,BGC,Qualifier
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
DOY,NA,NA,NA,DoY,NA
H_F,HEAT,Sensible heat turbulent flux (no storage correction),W m-2,H_f,Gap-filled variable
H_PI,HEAT,Sensible heat turbulent flux (no storage correction),W m-2,H_orig,Provided by PI
H_QC,HEAT,Sensible heat turbulent flux (no storage correction),W m-2,H_fqc,Quality control flag
LE_F,HEAT,Latent heat turbulent flux (no storage correction),W m-2,LE_f,Gap-filled variable
LE_PI,HEAT,Latent heat turbulent flux (no storage correction),W m-2,LE_orig,Provided by PI
LE_QC,HEAT,Latent heat turbulent flux (no storage correction),W m-2,LE_fqc,Quality control flag
NEE_F,PRODUCTS,Net Ecosystem Exchange,µmolCO2 m-2 s-1,NEE_f,Gap-filled variable
NEE_PI,PRODUCTS,Net Ecosystem Exchange,µmolCO2 m-2 s-1,NEE_orig,Provided by PI


In [22]:
# save
write.table(
  vars_all, "data/reddyproc_ameriflux_bgc_variables.tsv", row.names = FALSE,
  sep = "\t", quote = FALSE
)